<a href="https://colab.research.google.com/github/9StoneData/FUZZY_WUZZY/blob/main/DISP_2_MAN_MAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install python-Levenshtein
!pip install fuzzywuzzy
!pip install num2words


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
global CORES
import multiprocessing
CORES=multiprocessing.cpu_count()
import re

from num2words import num2words
import pandas as pd
pd.set_option('display.max_rows', 500)
import concurrent.futures
import datetime as dt
import time
import numpy as np
import sys
import gc
import random
import os
import pandas as pd
from os import listdir
from os.path import isfile, join

from fuzzywuzzy import process,fuzz

# Problem Statement:

Each day, the Leafly Project pulls over 90,000 product listings.  The products have inconsistent formats: "a b c vs ab c vs cb a" , etc.  The data is only valuable if it can be analyzed.  Also, it is my intention to monetize this data and build out a dashboard SaSS.  I need to store it in database, and I want to have clean tables.  

Solution:

I have been refining the "Humpty Dumpty" or for the sake of this writing, "Humpty Data" approach.  Looking back at the process, this makes me laugh.  As in the nursery ryhme, Humpty Dumpty fell off the wall.  However, with my relentless know enough to be dangerous approach, I pushed the data off the wall first.  

Here is how...

At first, I saw that Leafly also listed products separately from the dispensaries.  So, lets take a few steps back to give ourselves some room for a running start.  I decided to build a scraper to gather those listings thinking I could just match them up with some sort of python magic.

As I am about to take off with a full head of steam, I am stopped by a wizard from Levenshtein-ville.  Would you like to like to use this magic I have to help with your quest?  A yeah!!!  Here I go with a full head of steam and a new python weapon, the fuzzywuzzy library, and I am going to conquer this problem!!!

And I hit "Humpty Data" with everyting my stubborn mind had.  What is I compare this name to that one.  Now I need to asynchrnously process this...  Look its working... Its alive.  And now, its gone.  Play montouge....  I am back, lets do this again.  Now lets try weighting the searchers, load the names with aditional information... No, lets try building a parsrer with a library I never even used before.  Two days later, I discoverd gold in them there hills.  Cascadeing parser.  Cleans the names and captures key data.  Ok, I am about to get Mr. "Humpty Data" off the wall.  

Refector some code, fight off a bug or 100, and BOOM!  I have my "clean list", Humpty Data has gone down.... so I thoght.  I over shot the mark and just created more work.   I reduced the names down to similar names, but now I had grouped them too well.  Turns out you need the weights and ratios in the name... Who knew.  Here I am looking over my Data all over the creek bed as it fell of the bridge, thinkning to my self what had I done.

This time all the King's horses and all of the King's men will put Humpty Data back together again...



# STEP 1: Clean formatting

1. Remove manufacturer names from the products
2. Parse common "TAGS" out of the product name

Call it The Humpty Dumpty algorithim.  But this time all of the King's horses and all of the King's men could put Humpty Dumpty back together again!  

In [11]:

"""
Each day, one final scraper data file is created and needs to be processed to upoad to DB.  
"""
SDF =pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_.csv")
#clean out nan's
SDF['Manufacturer']=SDF['Manufacturer'].fillna("NOT GIVEN")
SDF['Product']=SDF['Product'].fillna("~")
SDF['Product_size']=SDF['Product_size'].fillna("*")

#build search ter3Clean out duplicates
SDF=SDF.drop_duplicates(subset='Product_Url',keep='first')
SDF=SDF.reset_index()
SDF=SDF.drop(['index'],axis=1)

#Due to inconistent format of how Dispensaries list their products on Leafly, we need to deconstructt the names, scrub it, and put it back together.
#This should / will clean the data set and allow for more accurate analysis 
#The product was split between name and size:  many had tags () in the name :  these are the product sizes
SDF['query_dirty']=SDF['Product']+" "+SDF['Product_size']
SDF['query_dirty']=SDF['query_dirty'].str.replace('|',' ',regex=True)
SDF['query_dirty']=SDF['query_dirty'].str.replace('(',' ',regex=True)
SDF['query_dirty']=SDF['query_dirty'].str.replace(')',' ',regex=True)
SDF['query_dirty']=SDF['query_dirty'].str.replace('-',' ',regex=True)
SDF['query']=SDF['query_dirty'].str.lower()

#as part of practive, if I make a chnage to a column, I add a column instead of replacing it.  I try too at least
SDF['Manufacturer_chk']=SDF['Manufacturer'].str.replace('*','',regex=True)
SDF['Manufacturer_chk']=SDF['Manufacturer'].str.replace('-','',regex=True)
SDF["TAG"]=""
start_size=(len(set(SDF['query'])))
products=list(SDF['query'])
corrected=0
updated=0
adjusted=0
t1=0
sizes=0
total_changed=0
for i,product in enumerate(products):
  test=product
  product=product.lower().replace('*flash sale*','')
  maker=SDF.at[i,'Manufacturer_chk'].lower().replace(".","").replace('  ',' ').replace(' AZ','').strip('*')
  TEST=True

  #Cascade :
  original=product
  #TEST1
  MATCHED=False
  if len(product)>len(maker):
    test_maker=maker.split()
    test_product=product.split()
    matches=[]
    for m,mk in enumerate(test_maker):
      for pr,prd in enumerate(test_product):
        if mk=="grön" and prd=="gron":
          matches.append("gron")
          MATCHED=True
        if prd in ["alien","item"] and pr<len(test_product) and test_product[pr+1] in ["labs","nine"]:
          matches.append("alien labs")
          MATCHED=True
        elif mk.replace("'","").replace(".","")==prd.strip(":").replace("'","").replace(".","").replace('!',''):
          matches.append(prd)
          MATCHED=True
          break
    for mt in matches:
      product=product.replace(str(mt),'',1)
    product=product.lstrip(" ").lstrip(":").replace("  "," ").rstrip(" ")
    if MATCHED:
      t1+=1
      total_changed+=1
  # if MATCHED and i%5==0:
  #   print(f"[p={i:5} {t1:5} {t1/(len(products))*100:2.2f}% {total_changed/(len(products))*100:2.2f}% ]{maker:20} ===> {product:60} {original:60}")

  
  units= {
    "Fractions"    :r"(\s{1}\d{0,4}\s{0,1}\d{1,4}\s{0,1}\/\s{0,1}\d{1,4}\s{0,1}(g|oz|mg))",
    "Ratios"       :r"((\d{1,5}\s{0,1}((thc:cbd|cbd:thc|thc:cbd:cbn|mg|g\s|thc|cbd|cbg|cbn){0,2}\s{0,1}){1,2})((\/|:)\s{0,1}\d{1,5}\s{0,1}((thc\s{0,1}:\s{0,1}cbd|cbd:thc|mg|g\s|thc|cbd|cbg|cbn){0,2}\s{0,1}){1,2}){1,4})",
    "Pack_size"    :r"(((\d{1,4}\s{0,1}(pk|pack|pieces|piece|Pieces|Piece|Bottle|bottle))|(\s\d{0,3}\s{0,1}\.{0,1}\d{0,2}(g\s|mg\s|oz\s))\s{0,1}x\s{0,1}\d{1,4})|((\s\d{0,3}\s{0,1}\.{0,1}\d{0,2}x\.{0,1}\s{0,1}\d{1,4}(g|mg|oz|ml))\s{0,1}))",
    'Weight'       :r"(\d{0,5}\s{0,1}[/.]{0,1}\s{0,1}\d{1,5}(mg|ml|g|oz)\s{0,1})",
    'Category_TAG' :r"((aio|jar|jars|mylar|vape|cart|flower|ridge|budder|cartridge|hybrid|\sh\s|\si\s|\ss\s|\si\/h|\s\/i|cartridges|preroll|gummy|gummies|preerolls|pods|pod|pre pack|pre roll|pre rolls|live|resin|rosin|tablets|hybrid|sativa|indica){1,10})"

  }
  #I know: ugly but do not want to spend an hour redoing regex
  product=" "+product+" " 
  for k,v in units.items():
    result = re.findall(v, product)  
    original = product
    if result:
      for res in result:
        product=product.replace(res[0],' ',1)
        SDF.at[i,"TAG"]+=f"+{k}<{res[0].replace(' ','').replace('pack','pk')}>"
        SDF.at[i,'BASE_NAME']=product.replace("[","").replace("]","").replace(":","").replace("  "," ").replace("*","").replace("'",'').replace(',','').lstrip(",").lstrip(" ").rstrip(" ").lstrip(',')
        if k!='Fractions':
          SDF.at[i,k]=res[0].replace(' ','').replace('pack','pk')
        else:
          SDF.at[i,"Weight"]=res[0].replace(' ','').replace('pack','pk')
        
      if len(SDF.at[i,'BASE_NAME'])==0:
          SDF.at[i,'BASE_NAME']="123456789987654321"
          #print(f"{i:5} {k:20} {res[0]:20} >><<< {original:60}<<======>>{SDF.at[i,'BASE_NAME']:25} ------------------>  {len(SDF.at[i,'BASE_NAME'])}")
      if i%20==0:
         print(f"{i:5} {k:20} {res[0]:20} >><<< {original:60}<<======>>{SDF.at[i,'BASE_NAME']:25} ------------------>  {len(SDF.at[i,'BASE_NAME'])}")

        
end_size=(len(set(SDF['query'])))

print(f"INitially there wher {start_size} unique product names\nAfter step one, there are {end_size} unique names ")
SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S1_complete.csv",index=False)

Streaming output truncated to the last 5000 lines.
15480 Weight                1g                  >><<<  wax g4 x thin mints 1g i                                   <<======>>wax g4 x thin mints i     ------------------>  21
15480 Category_TAG          i                   >><<<  wax g4 x thin mints i                                      <<======>>wax g4 x thin mints       ------------------>  19
15500 Ratios               1:1 thc              >><<<  premium 1:1 thc/cbd vape pod mango .5g h                   <<======>>premium /cbd vape pod mango .5g h ------------------>  33
15500 Weight                .5g                 >><<<  premium  /cbd vape pod mango .5g h                         <<======>>premium /cbd vape pod mango h ------------------>  29
15500 Category_TAG          h                   >><<<  premium  /cbd vape pod mango h                             <<======>>premium /cbd   mango      ------------------>  20
15520 Weight                1g                  >><<<  live resin b

# STEP 2: Clean Manufacturer names

Manufacturers are listed with slight variations to their name.  Same problem as the products.  Using the Fuzzy library, I am going to macth mismatched Manufacturers.  Dunes = duns= dUNEs... etc


In [17]:
def process_for_match(LIST_o_DF):

  try:

    QUERY_DF          = LIST_o_DF[0]
    q1_LIST_column    = QUERY_DF.at[0,'query_column']
    q1_LIST           = list(QUERY_DF[q1_LIST_column]) 

    SEARCH_DF         = LIST_o_DF[1]
    s1_LIST_coliumn   = SEARCH.at[0,'search_column']
    s1_LIST           = list(set(SEARCH_DF[s1_LIST_coliumn]))

    VC                = LIST_o_DF[2]
  except:
    print("error initializing dfs")

  try:
    if QUERY_DF.at[0,'algo_step']==1:
      for q,query in enumerate(q1_LIST):
        
        try:
          ops=process.extract(query,s1_LIST)
        except:
          print("oops error",query)

        master=query
     
        if len(ops)>0:
          
          for o,op in enumerate(ops):
            EXACT=False
            #assuming forst one is the same, but can check
            if query==op[0]:
              EXACT=True
        
            #ran various thresholds.  92 + is least number of false positives
            if not EXACT and  op[1]>=92:
              #Decide on master name by which one hsows up more 
              man1=query
              man2=op[0]
    
              if VC.at[man2,'Manufacturer'] > VC.at[man1,'Manufacturer']:
                master=man2
              print(f"{q:3} {query:40} =?= {op[0]:40} {VC.at[man1,'Manufacturer']:5} {VC.at[man2,'Manufacturer']:5} {master}")
              QUERY_DF.at[q,'Multiple_Possible_Names']=True
              QUERY_DF.at[q,f"_{o}_possible_name"]=op[0]
              QUERY_DF.at[q,"Number_of_Possible_Names"]=o
              QUERY_DF.at[q,'MASTER_MAN_NAME']=master
              
              
        
        # if q%10==0:
        #   print(ops)
      #by design, as the DF's are returned, the master df is rebuilt
  except:
    print("Error getting ops")

  #was returning a tuple for other steps so i coudl reuse this function.  just putting
  #a not here so when this bites me in the ass, i know why!!!
  return(QUERY_DF)

def process_for_match_THREADED(MASTER_LIST):
  threads=len(MASTER_LIST)

  FDF=pd.DataFrame()
  FMMDF=pd.DataFrame()

  with concurrent.futures.ProcessPoolExecutor(max_workers=threads) as executor:
    try:
        for r,results in enumerate(executor.map(process_for_match,MASTER_LIST)):
          FDF=FDF.append(results,ignore_index=True)
        print("done done done@@@@@@@@@@@@@@@@@@@@@@@@@@")
        path=FDF.at[0,'directory']
        file_name=FDF.at[0,'query_name']
        file_date_stamp=dt.datetime.now()
        file_date_stamp=file_date_stamp.strftime("%m_%d_%Y_%H_%M_")
        FDF.to_csv(join(path,f"_{file_name}"),index=False)
        FDF.to_csv(join(path,f"_{file_date_stamp}_{file_name}"),index=False)
        print(FDF.info())

    except:
        print("OOPS")

  return FDF




SDF =pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S1_complete.csv")
print(SDF.shape)

#altering the column to cath mmore consitent matches.
SDF['Manufacturer_Scraped']=SDF['Manufacturer']
SDF['Manufacturer']=SDF['Manufacturer'].str.lower().replace(" ","")


#list of manufacturers
disp_mans=list(set(SDF['Manufacturer']))
mans=list(set(SDF['Manufacturer']))


#DF lookup table for competing names
VC=pd.DataFrame(SDF['Manufacturer'].value_counts())

#depending on cpu use, Process pool or Thread pool will be used for asynchronous processing
#keep in mind that a=b in one thread, b=a in another or in a different search.  resolve this
# by picking the one with more products as the master name.  will require manual review of changes
# but, can autmate the process with screen input once out of the step


#Prepair data for asynchrounous processing
THREADS=CORES

query  = disp_mans.copy()
search = mans.copy()

QUERY  = pd.DataFrame(query ,columns=['Manufacturer'])
SEARCH = pd.DataFrame(search,columns=['Manufacturer'])

#configure 
QUERY['THREAD']=QUERY.index%THREADS
MASTER_LIST=[]
for i in range(THREADS):
  temp  = []
  tdf   = QUERY[QUERY['THREAD']==i]
  #send copies not pointers to original
  tdf   = tdf.copy()
  #easier to reference rows
  tdf.reset_index(inplace=True)
  tdf.drop(['index'],axis=1,inplace=True)
  tdf['query_column'] = 'Manufacturer'
  tdf['query_name']="product_manufacturer_vs_vendor_manufacturer_.csv"
  tdf['directory'] = "/content/drive/MyDrive/LEAFLY_Project/Data/THREAD_RESULTS/MAPPING"
  tdf['Multiple_Possible_Names']=False
  tdf['MASTER_MAN_NAME']=tdf['Manufacturer']
  tdf['algo_step']=1
  SEARCH = SEARCH.copy()
  SEARCH['search_column']='Manufacturer'
  temp.append(tdf)
 
  temp.append(SEARCH)
  VC=VC.copy()
  temp.append(VC)
  MASTER_LIST.append(temp)


FDF=process_for_match_THREADED(MASTER_LIST)

FDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_matched_MAN_key.csv",index=False)
SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S2_complete.csv",index=False)

(89730, 57)
  0 sprinkles                                =?= sprinkle                                     8   189 sprinkle
  0 everest                                  =?= az everest                                   1     5 az everest
  0 puraearth                                =?= pura earth                                 249    11 puraearth
  0 moonrock                                 =?= moonrocks                                    2     2 moonrock
  0 budsbros                                 =?= budbros                                      2    16 budbros
  0 core concentrates                        =?= core concentrate                            39   166 core concentrate
  0 budsbros                                 =?= budsbro                                      2     2 budbros
  0 white mountain health center (zazzo)     =?= white mountain health center                 6    12 white mountain health center
  0 desert medical campus inc. (payson)      =?= desert medical campus,

  8 mindryte                                 =?= mind ryte                                   35   108 mind ryte
  8 mohave canabis co                        =?= mohave cannabis co                           2    26 mohave cannabis co
  8 sprinkle                                 =?= sprinkles                                  189     8 sprinkle
  8 mohave canabis co                        =?= mohave cannabis co.                          2   110 mohave cannabis co.
  7 pax®                                     =?= pax                                         73    18 pax®
  9 trumed                                   =?= tru|med                                     12    12 trumed
  9 alien labs                               =?= alienlabs                                  439    27 alien labs
  9 skunkbrand                               =?= skunk brand                                  1     1 skunkbrand
  8 patient care center 301, inc             =?= patient care center 301, inc. (retail)     

Step 3:  Fisrt clean up mess.  FDF from previous cell needs to be written to a file.  2.  Do not read in fresh scrape anymre.  now you read in results from step 1.

In [19]:
#Original Product Scrped Table = MAN_PROD
#reloading for the sake of not having to scroll back to top to run again
#MAN_PROD=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_PROD_PAGE_DETAILS_.csv")
#SDF =pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_.csv")
SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S2_complete.csv")
FDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_matched_MAN_key.csv")
#output from last cell....
FDFa = FDF[['Manufacturer','MASTER_MAN_NAME','Number_of_Possible_Names']]
print(f" Number of unique Manufactureres___________: {FDFa.shape[0]:4}")
#FDFa=FDFa.drop_duplicates(subset=['MASTER_MAN_NAME'],keep='first')

temp = FDFa[FDFa['Manufacturer']!=FDFa['MASTER_MAN_NAME']]
print(f" Manufacturers to correct__________________: {temp.shape[0]:4}")


tofix=0
for man in list(temp['Manufacturer']):
  test=SDF[SDF['Manufacturer']==man]
  tofix+=len(test)

corrections=0
for man in list(temp['Manufacturer']):
  tdf=(SDF.loc[SDF['Manufacturer']==man,'Manufacturer'])
  corrections+=len(tdf)
  SDF.loc[SDF['Manufacturer']==man,'Manufacturer']=temp.loc[temp['Manufacturer']==man,'MASTER_MAN_NAME']
  #tdf=(MAN_PROD.loc[MAN_PROD['Manufacturer']==man,'Manufacturer'])
  # print(f"{man} :: {len(tdf)} :: {temp.loc[temp['Manufacturer']==man,'MASTER_MAN_NAME']} ::::::::")
  #print(f"{temp.loc[temp['Manufacturer']==man,'MASTER_MAN_NAME']} ::::::::")

errors=0
for man in list(temp['Manufacturer']):
  test=SDF[SDF['Manufacturer']==man]
  if len(test)>0:
    print(man)
  errors+=len(test)
print(f" Products with Manufacturer name adjusted__: {corrections:4} of {tofix} with {errors} missed")

SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S3_complete.csv",index=False)
FDFa.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_keys.csv",index=False)

 Number of unique Manufactureres___________:  883
 Manufacturers to correct__________________:   66
 Products with Manufacturer name adjusted__:  485 of 485 with 0 missed


# STEP 4: Match base names and clean formats to fit better after

In [22]:
def parse_products(LIST_o):
  """
  this could be built out to overwrite other columns to fill in blanks
  """
  #Find numbers
  find_numbers=r"((\.){0,1}\d{1,4}\.{0,1}\d{0,2})"


  To=time.time()
  DFO=LIST_o[0]
  ML=LIST_o[1]    #df of all product counts 
  s1_LIST=list(ML['index'])
  #print(DFO.shape)
  thread=DFO.at[0,'THREAD']
  RDF=pd.DataFrame()
  #Index(['index', 'query', 'THREAD']
  products=list(DFO['index'])


  for p,product in enumerate(products):
    
    tdf=DFO[DFO['index']==product]
    tdf=tdf.copy()
    tdf=tdf.reset_index()
    tdf=tdf.drop(['index'],axis=1)
    tdf.at[0,'product']=product
    tdf.at[0,'DUP_PRODUCT_NAME']="NO"
    TRY=True
    MATCHED=False
    total=0
    NEW=product
    try:
      
      try:
        if product!="123456789987654321":
          ops=process.extract(product,s1_LIST)
        else:
          TRY=False
      except:
        print("ops error9999999999999999",product)
      
      try:
      
        if TRY:
          for o,op in enumerate(ops):
            if op[1]>=85:
              total+=1
              product1=product.replace(" ","")
              product2=op[0].replace(" ","")
              product1=product.replace(":","_to_")
              product2=op[0].replace(":","_to_")


              nop=process.extractOne(product1,[product2])
              token=fuzz.token_set_ratio(product1,product2)
              ratio=fuzz.ratio(product1,product2)
              part_rat=fuzz.partial_ratio(product1,product2)

              avg=(nop[1]+token+ratio+part_rat)/4
              v1=v2=0
              if total>1:
            
                tdf.at[0,'DUP_PRODUCT_NAME']="YES"
                v1=ML.loc[ML['index']==product,'BASE_NAME']
                v1=v1.iloc[0]
                v2=ML.loc[ML['index']==op[0],'BASE_NAME']
                v2=v2.iloc[0]
                v1o=v1
                v2o=v2
                if v1>10 and v2>10:
                  MATCHED=False
                elif avg>86 :
                  MATCHED=True
                  if v2 >= v1:
                    v1=v2
                    NEW=op[0]
                  else:
                    NEW=product
                  # if p%3==0:
                  #   print(f"-------------------------------------------------------\
                  #   \nProduct[query ]: {product:65}  vs\
                  #   \nProduct[search]: {op[0]:65} possibles Scores:[{p:5} {op[1]:4} :: {nop[1]:4} {token:4} {ratio:4} {part_rat:4}:::AVERAGE: {avg:3.2f}] Counts:[{v1o:4} {v2o:4} {v1o/v2o:2.2f}]")

          
        try:    
          if MATCHED:
            tdf.at[0,'MASTER_PRODUCT_NAME']=NEW.replace("[","").replace("]","").replace(":","").replace("    "," ").replace("   "," ").replace("  "," ").replace("*","").replace("'",'').replace(',','').lstrip(",").lstrip(" ").rstrip(" ").lstrip(',')  
          else:
            tdf.at[0,'MASTER_PRODUCT_NAME']=product.replace("[","").replace("]","").replace(":","").replace("    "," ").replace("   "," ").replace("  "," ").replace("*","").replace("'",'').replace(',','').lstrip(",").lstrip(" ").rstrip(" ").lstrip(',')
          RDF=RDF.append(tdf,ignore_index=True)
        except:
          print("printing errorr00000000000",MATCHED,product)
        # if p %5==0:
        #   print(f"[{(p+1)/len(products)*100:3.2f}% Complete] In thread {thread:3} on {p:5} of {len(products):5} products: {product:50} with {total:3} ")
        try:
          test=RDF.loc[RDF['DUP_PRODUCT_NAME']=="YES",:]
          if p%5==0:
            Total_TIME=time.time()-To
            EST_Total_Time=Total_TIME/((p+1)/len(products))


            print(f"[{(p+1)/len(products)*100:6.2f}% Complete] [{len(test)/(p+1)*100:6.2f}% Duplicate] [{Total_TIME/3600:6.2f}Hrs {(EST_Total_Time-Total_TIME)/3600:4.2f} hrs left]==>> In thread {thread:3} on {p:5} of {len(products):5} product: {product:50} :: {total:3} ")
        except:
          print("error getting test",product)
      
      except:
        print("error processing ops",product)
    
    except:
      print("oops error",product)
    # if len(tdf)<1:
    #   print("ohhhhhhhhhhhhhhhhhh")
    # print(tdf.shape)
    # DFO=pull_uoms(product,p,SDF)
    # else:
    #   query=tdf.at[0,'query']
    #   pull_uoms(query,p,tdf)
    #   RDF=RDF.append(tdf,ignore_index=True)

  return(RDF)


def MASTER_LIST_parse_products(THREADS):
  SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S3_complete.csv")
  print(SDF.columns)


  # SDF=SDF[['Dispensary','Product_size', 'Manufacturer', 'Category', 'Listed_content', 'THC%',
  #       'CBD%', 'UOM', 'Price', 'query','Manufacturer_chk','Product_Url','mg',
  #       'g', 'ratio', 'thc_cbd', 'pk_cont', 'pk', 'strain', 'oz']]

  
  print(SDF.shape)       
  print(SDF.columns)

  SDF=SDF.drop_duplicates(subset="Product_Url",keep='first')
  SDF=SDF[~SDF['BASE_NAME'].isna()]
  SDF = SDF.dropna(subset=['Manufacturer','Product_Url','BASE_NAME'])
  SDF=SDF.astype(str)
  """
  Thought about "overweighting the name to reduce 1000 to 500 matches, but caused more noise 
  """
  SDF['BASE_NAME']=SDF['BASE_NAME']#+" :: "+SDF['Manufacturer']

  SDF=SDF.reset_index()
  SDF=SDF.drop(['index'],axis=1)

  SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S4_complete.csv",index=False)
  print(SDF.shape)
  #SDF['query']=SDF['Manufacturer']+" "+SDF['query']
  VC=pd.DataFrame(SDF['BASE_NAME'].value_counts())
  VC=VC.reset_index()
  #make products the index
  VC['THREAD']=VC.index%THREADS
  MASTER_LIST=[]
  for i in range(THREADS):
    temp=[]
    tdf=VC[VC['THREAD']==i]
    print(tdf.shape,VC.shape,len(MASTER_LIST))
    tdf=tdf.copy()
    tdf=tdf.reset_index()
    #not index but level_0 since did not drop index last time
    tdf=tdf.drop(['level_0'],axis=1)
    tVC=VC.copy()
    temp.append(tdf)
    temp.append(tVC)
    MASTER_LIST.append(temp)
  print(tVC.columns) 
  return(MASTER_LIST)

def parse_products_THREADED(MASTER_LIST):
  threads=len(MASTER_LIST)
  FDF=pd.DataFrame()
  FMMDF=pd.DataFrame()
  with concurrent.futures.ProcessPoolExecutor(max_workers=threads) as executor:
    try:
        for r,results in enumerate(executor.map(parse_products,MASTER_LIST)):
          FDF=FDF.append(results,ignore_index=True)
        print("TESTING")
        # path=FDF.at[0,'directory']
        # file_name=FDF.at[0,'query_name']
        # file_date_stamp=dt.datetime.now()
        # file_date_stamp=file_date_stamp.strftime("%m_%d_%Y_%H_%M_")
        # FDF.to_csv(join(path,f"_{file_name}"),index=False)
        # FDF.to_csv(join(path,f"_{file_date_stamp}_{file_name}"),index=False)
        print(FDF.info(),"\nFULL DF")

    except:
        print("OOPS")
  return(FDF)


THREADS=CORES
MASTER_LIST=MASTER_LIST_parse_products(THREADS)
FDF=parse_products_THREADED(MASTER_LIST)
#queryy in the numbver of occurances
FDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_product_map_.csv",index=False)


Index(['Type', 'Name', 'Abbreviation', 'Capital', 'THREAD', 'Market_Focus',
       'DISPS_SRP', 'Pull_Date', 'Dispenary_base_leaf_url', 'Thread_List',
       'In_Market_Focus', 'Canada', 'Dispensary', 'Name_Tag', 'total_products',
       'Last_update', 'Member_Since', 'google_map', 'Total_Followers',
       'Address', 'Street', 'Town', 'State', 'Email', 'Phone', 'Website',
       'License', 'Leafly_review_count', 'FILE_NAME', 'total_pages',
       'products_on_last_page', 'Products_Page_URL', 'current_page', 'fop',
       'Page_Load_Tries', 'Product_Url', 'P_number', 'Variant', 'Product',
       'Product_size', 'Manufacturer', 'Category', 'Listed_content', 'THC%',
       'CBD%', 'UOM', 'Price', 'Collected_at', 'query_dirty', 'query',
       'Manufacturer_chk', 'TAG', 'BASE_NAME', 'Weight', 'Category_TAG',
       'Ratios', 'Pack_size', 'Manufacturer_Scraped'],
      dtype='object')
(89730, 58)
Index(['Type', 'Name', 'Abbreviation', 'Capital', 'THREAD', 'Market_Focus',
       'DISPS_SRP'

# STEP 5:  Adjust mismatched weights.  put all weights in mg vs g

Some clean up from step 1 where 10 pieces /500mg format not getting picked up properly... more debug time...requires some hacking in this step

In [24]:
SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S4_complete.csv")
SDF.shape
SDF.columns
SDF['Numerical_Weight']=0
SDF['Numerical_Weight_units']="unknown"
units={
        'mg' :r"(\d{0,5}\s{0,1}[/.]{0,1}\s{0,1}\d{1,5}(mg)\s{0,1})",
        'g'  :r"(\d{0,5}\s{0,1}[/.]{0,1}\s{0,1}\d{1,5}(g)\s{0,1})",
        'oz' :r"(\d{0,5}\s{0,1}[/.]{0,1}\s{0,1}\d{1,5}(oz)\s{0,1})"
      }
#
#need g and mg separate

for i in SDF.index:
  weight=str(SDF.at[i,'Weight'])
  for k,v in units.items():
    result=re.search(v,weight)
    fw=0
    unit="NO_LISTED"
    if result:
      try:
        fw=result.group().replace(k,'')

        if len(fw)>4:
          fw=fw[-4:]
        if len(fw.split("/"))>1:
          # print(fw.split("/"))
          fw=fw.split("/")
          if fw[0]=="":
            fw=float(fw[-1])
          else:
            if k=="oz":
              fw[0]=1
            fw=float(fw[0])/float(fw[-1])
        else:
          fw=float(fw.split("/")[-1])
        if k=="g":
          unit='mg'
          fw=fw*1000
        else:
          unit=k
      
      except:
        print(f"WEIGHT ERROR {weight} {SDF.at[i,'Product']}")
      
      
      SDF.at[i,'Numerical_Weight']=fw
      SDF.at[i,'Numerical_Weight_units']=k
      if i%5==0:  
        print(f"{weight:10} {fw:10} {unit:4} {SDF.at[i,'query']:50} {SDF.at[i,'BASE_NAME']:50}")
      
  # if not result:
  #   result=re.search(g_check,weight)
  #   if result:
  #       try:
  #         fw=float(result.group().replace('g','').replace("/",""))*1000
  #         unit="mg"
  #       except:
  #         print(f"WEIGHT ERROR {weight} {SDF.at[i,'Product']}")
  

  # if result:
  #   print(f"{weight:10} {fw:10} {unit:4} {SDF.at[i,'query']:50} {SDF.at[i,'BASE_NAME']:50}")
      
  #   #if result:print(weight,result.group().replace('g',''))
  # if not result:
  #   result=re.search(oz_check,weight)
  #   if result:print(weight,result.group().replace('oz',''))
SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S5_complete.csv",index=False)

Streaming output truncated to the last 5000 lines.
500mg           500.0 mg   fruit punch g] *                              fruit punch                                       
.25g            250.0 mg   gelato cream [.25g] *                              gelato cream                                      
100mg           100.0 mg   nightly relaxation [10pk] 100mg                    nightly relaxation                                
50mg             50.0 mg   sour blue raspberry [20pk] 50mg                    sour blue raspberry                               
1g             1000.0 mg   pai gow [1g] *                                     pai gow                                           
1g             1000.0 mg   banana hammock [1g] *                              banana hammock                                    
.5g             500.0 mg   berries [.5g] *                                    berries                                           
1g             1000.0 mg   white diagonal x sf wifi

# STEP 6: Adjust Scrape Categories part 1
  LOow Fruit: Hybrid flower to flower etc..

In [25]:
SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S5_complete.csv")
LPMK=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/LEAFLY_PRODUCT_CATEGORY_KEY.csv")
LPMK.columns
SDF['Main_Category']="FIND"

print("Before mapping, here are the different category counts:")
print(SDF.Category.value_counts())

sub_cat=list(LPMK['Sub_Category'])

for sub in sub_cat:
  SDF.loc[SDF['Category']==sub,"Main_Category"]=LPMK.loc[LPMK['Sub_Category']==sub,'Main_Category'].iloc[0]

SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S6_complete.csv",index=False)

print("After mapping, here are the different category counts:")
print(SDF.Main_Category.value_counts())


Before mapping, here are the different category counts:
Cartridge                 19410
Edible                    11653
Concentrate                9898
Preroll                    7460
Hybrid flower              6304
Other                      5056
Flower                     4078
Candies                    1986
Indica flower              1068
Sativa flower               735
Topical                     655
Pre rolls                   551
Infused                     479
Capsules                    457
Pre roll packs              450
Accessory                   442
Disposables                 357
Chocolates                  302
Cartridges                  239
Waxes                       176
Drinks                      160
Drink mixes                 149
Sublinguals                 140
Baked goods                 137
Infused packs                92
Rosins                       66
Rick simpson oil (rso)       48
Infused blunts               47
Mints                        46
Blunt packs     

# STEP 7: Start building it back.

Merge Scrape DF with the product keys and the manufacturer keys.  This should be step 5.  Works not perfect... move on




In [30]:
#Most current Scrape Data Frame.  It has the BASE_NAMES that merge with the key DF and manufacture
#names to merge with MM df
SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S6_complete.csv")
print(SDF.shape)
#Current Product map key DF.  have to rename colomns since I converted the SDF Value count df.
PMK=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_product_map_.csv")
PMK=PMK.rename(columns={'BASE_NAME':'NA','product':'BASE_NAME'})
SDF=pd.merge(SDF,PMK,on='BASE_NAME',how='inner')
print(SDF.shape)
#Currnt Manufacturer map key DG
MMK=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/DISP_PRODUCT_LSIT_keys.csv")
SDF=pd.merge(SDF,MMK,on='Manufacturer',how="inner")
print(SDF.shape)

SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S7_complete.csv",index=False)

(73035, 62)
(73035, 67)
(73035, 69)


STEP 8: Fill in the categories fromt the Categorytags collected

In [68]:
SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S7_complete.csv")

CAT_Start=len(SDF[SDF['Main_Category']=="FIND"])

PTPK=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Inputs/parsed_tag_product_key.csv",names=['Key','Main_Category'],header=None)

TAGS=list(PTPK['Key'])

WDF=SDF.loc[(SDF['Category']=='Other')&(~SDF['Category_TAG'].isna())]
WDF=WDF.copy()
master=""
WDF['Category_TAG']=WDF['Category_TAG'].astype(str)
for i in WDF.index:
  tags=WDF.at[i,'Category_TAG'].split('+')
  for tag in tags:
    if tag in TAGS:
      SDF.at[i,'Main_Category']=PTPK.loc[PTPK['Key']==tag,'Main_Category'].iloc[0]
  
  
  
  if i%10000==0:
    print(i)

CAT_End=len(SDF[SDF['Main_Category']=="FIND"])

print(f"Before mapping tags, there where {CAT_Start} products that needed categories\n\
After mapping tags, there are {CAT_End} products that need categories")

SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S8_complete.csv")

Before mapping tags, there where 5056 products that needed categories
After mapping tags, there are 3288 products that need categories


STEP 9: Start to fill in missing categories. Should be able to filter down to A or b with an obvious choice.  Start with master names, and slowly build them back filloint in along the way. (two bites????)  THIS IS A BIG TASK.  MAY NEED TO MULTI PROCESS

one block at a time: start with simple eitheor or choices.  if 50 are Flower and 2 are missing, I am going to assign it to flower

In [70]:
SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S8_complete.csv")
SDF['MASTER_PRODUCT_NAME_MAN']=SDF['Manufacturer']+'+'+SDF['MASTER_PRODUCT_NAME']
CAT_Start=len(SDF[SDF['Main_Category']=="FIND"])

WDF=SDF[SDF['Main_Category']=="FIND"]

for i in WDF.index:
  test=SDF.loc[SDF['MASTER_PRODUCT_NAME_MAN']==SDF.at[i,'MASTER_PRODUCT_NAME_MAN'],:]
  tdf=pd.DataFrame(test['Main_Category'].value_counts())
  
  if len(tdf)==2:
    #By design it is either FIND or cat.  
    CATS=list(tdf.index)
    for CAT in CATS:
      if CAT!="FIND":
        break
    SDF.loc[SDF['MASTER_PRODUCT_NAME_MAN']==SDF.at[i,'MASTER_PRODUCT_NAME_MAN'],'Main_Category']=CAT 

CAT_End=len(SDF[SDF['Main_Category']=="FIND"])
print(f"Before mapping tags, there where {CAT_Start} products that needed categories\n\
After mapping tags, there are {CAT_End} products that need categories")

SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S9_complete.csv",index=False)

Before mapping tags, there where 3288 products that needed categories
After mapping tags, there are 926 products that need categories


Step 9.2 Add weights and same logic

In [71]:
SDF.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Type', 'Name', 'Abbreviation', 'Capital',
       'THREAD_x', 'Market_Focus', 'DISPS_SRP', 'Pull_Date',
       'Dispenary_base_leaf_url', 'Thread_List', 'In_Market_Focus', 'Canada',
       'Dispensary', 'Name_Tag', 'total_products', 'Last_update',
       'Member_Since', 'google_map', 'Total_Followers', 'Address', 'Street',
       'Town', 'State', 'Email', 'Phone', 'Website', 'License',
       'Leafly_review_count', 'FILE_NAME', 'total_pages',
       'products_on_last_page', 'Products_Page_URL', 'current_page', 'fop',
       'Page_Load_Tries', 'Product_Url', 'P_number', 'Variant', 'Product',
       'Product_size', 'Manufacturer', 'Category', 'Listed_content', 'THC%',
       'CBD%', 'UOM', 'Price', 'Collected_at', 'query_dirty', 'query',
       'Manufacturer_chk', 'TAG', 'BASE_NAME', 'Weight', 'Category_TAG',
       'Ratios', 'Pack_size', 'Manufacturer_Scraped', 'Numerical_Weight',
       'Numerical_Weight_units', 'Main_Category', 'level_0', 'NA', 'TH

In [78]:
SDF=pd.read_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S9_complete.csv")
SDF['MASTER_PRODUCT_NAME_MAN_WEIGHT']=SDF['MASTER_PRODUCT_NAME_MAN']+"+"+SDF['Weight']
CAT_Start=len(SDF[SDF['Main_Category']=="FIND"])

WDF=SDF[SDF['Main_Category']=="FIND"]

for i in WDF.index:
  test=SDF.loc[SDF['MASTER_PRODUCT_NAME_MAN_WEIGHT']==SDF.at[i,'MASTER_PRODUCT_NAME_MAN_WEIGHT'],:]
  tdf=pd.DataFrame(test['Main_Category'].value_counts())
  
  if len(tdf)==2:
    #By design it is either FIND or cat.  
    CATS=list(tdf.index)
    for CAT in CATS:
      if CAT!="FIND":
        break
    SDF.loc[SDF['MASTER_PRODUCT_NAME_MAN_WEIGHT']==SDF.at[i,'MASTER_PRODUCT_NAME_MAN_WEIGHT'],'Main_Category']=CAT 

CAT_End=len(SDF[SDF['Main_Category']=="FIND"])
print(f"Before mapping tags, there where {CAT_Start} products that needed categories\n\
After mapping tags, there are {CAT_End} products that need categories, \n\
Now there are {len(set(SDF['MASTER_PRODUCT_NAME_MAN_WEIGHT']))} unique products while at the begining\n\
There where {len(set(SDF['Product']))} ")

SDF.to_csv("/content/drive/MyDrive/LEAFLY_Project/Data/Run_Results/9Stone/_AZ_Market_Data_S9_2_complete.csv",index=False)

Before mapping tags, there where 926 products that needed categories
After mapping tags, there are 815 products that need categories, 
Now there are 15015 unique products while at the begining
There where 22466 


STEP 9.3 Now add back ratios

Refactor into one step 

In [76]:
SDF[SDF['Main_Category']=="FIND"]['MASTER_PRODUCT_NAME_MAN_WEIGHT'].head(100)

2552                       tru infusion+cbd capsules+300mg
2553                       tru infusion+cbd capsules+600mg
2554                       tru infusion+cbd capsules+300mg
2555                       tru infusion+cbd capsules+600mg
3464                                                   NaN
3606                       tru infusion+alien grapevine+1g
3630                        tru infusion+jack herer+1000mg
3631                        tru infusion+jack herer+1000mg
3711               tru infusion+pet cbd dog biscuits+100mg
3712               tru infusion+pet cbd dog biscuits+100mg
3863                tru infusion+cbd freeze roll on+1000mg
3864                tru infusion+cbd freeze roll on+1000mg
3865                tru infusion+cbd freeze roll on+1000mg
3866                tru infusion+cbd freeze roll on+1000mg
3876                tru infusion+relief salve stick+1000mg
3877                tru infusion+relief salve stick+1000mg
3880           tru infusion+broad spectrum tincture+1600